In [19]:
# Setup
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    print("⚠️ Warning: OPENAI_API_KEY not found. Set it in .env file.")
else:
    print("✅ API key loaded successfully")

✅ API key loaded successfully


In [20]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


# Load PDF
loader = PyPDFLoader("About_Reuben.pdf")
documents = loader.load()   # returns List[Document]

In [22]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Create splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=600,
    chunk_overlap=60,
    length_function=len
)

# Split documents
chunks = text_splitter.split_documents(documents)

print(f"Split {len(documents)} documents into {len(chunks)} chunks")
for i, chunk in enumerate(chunks[:3]):
    print(f"\nChunk {i+1}: {chunk.page_content}")

Split 2 documents into 8 chunks

Chunk 1: About Me - Reuben Mulero
I  am  Reuben  Mulero,  a  graduate  in  Entrepreneurship  (Entrepreneurial  Studies) from  the  Federal
University of Agriculture, Abeokuta (FUNAAB), and a passionate full-stack AI developer. My journey has
been defined by a relentless curiosity, a strong problem-solving mindset, and a dedication to creating real-
world solutions through technology and innovation.
From the beginning, I have always been drawn to understanding systems and building solutions that have
tangible impact. My academic and professional experiences have equipped me with a unique combination

Chunk 2: of technical skills, leadership abilities, and business insight.
Technical Expertise and Projects
I specialize in full-stack development, building both the frontend and backend of web applications with a
strong focus on usability, scalability, and performance. My technical toolkit includes: - Frontend: React, Vite,
Tailwind CSS - Backend: FastAPI - 

In [23]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.docstore.document import Document
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

# Convert documents
lc_docs = chunks

# Vector store (Chroma)
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    openai_api_key=openai_api_key
)


# persist_directory allows saving DB locally; optional
vectorstore = Chroma.from_documents(
    lc_docs,
    embeddings,
    collection_name="my_rag_collection",
    persist_directory="./chroma_db"  # optional
)

retriever = vectorstore.as_retriever(search_kwargs={"k": 4}
)

# LLM
llm = ChatOpenAI(
    model="gpt-3.5-turbo",
    temperature=0,
    openai_api_key=openai_api_key
)

# Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert assistant. Answer ONLY using the provided context. "
     "If the answer is not in the context, say it is not available."
    ),
    ("human",
     "Question: {question}\n\nContext:\n{context}"
    )
])


# Build RAG pipeline
rag_chain = (
    RunnableParallel(context=retriever, question=RunnablePassthrough())
    | prompt
    | llm
)

# Query
response = rag_chain.invoke("tell me about Reuben mulero?")
print(response)


content='Reuben Mulero is a builder by instinct and a problem-solver by choice.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 448, 'total_tokens': 465, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CmfNwFk23ub0CEbMjrG49nwv8gmO7', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b1cce-84f4-7892-9d00-86470e628035-0' usage_metadata={'input_tokens': 448, 'output_tokens': 17, 'total_tokens': 465, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [25]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

# Store histories
chat_store = {}

def get_session_history(session_id: str):
    if session_id not in chat_store:
        chat_store[session_id] = InMemoryChatMessageHistory()
    return chat_store[session_id]

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Prompt
conv_prompt = ChatPromptTemplate.from_messages([
    ("system",
     "You are an expert assistant. Answer ONLY using the provided context. "
     "If the answer is not in the context, say you do not know."
    ),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "Context:\n{context}\n\nQuestion: {question}")
])

# Base chain
conv_chain_base = (
    RunnableParallel(
        context=lambda x: format_docs(retriever.invoke(x["question"])),
        question=lambda x: x["question"],
        chat_history=lambda x: []  # REQUIRED placeholder
    )
    | conv_prompt
    | llm
    | StrOutputParser()
)

# History wrapper
conv_chain = RunnableWithMessageHistory(
    conv_chain_base,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history"
)

print("✅ Conversational RAG chain ready")


✅ Conversational RAG chain ready


In [26]:
response = conv_chain.invoke(
    {"question": "Who is Reuben Mulero?"},
    config={"configurable": {"session_id": "user-1"}}
)

print(response)


Reuben Mulero is a builder by instinct and a problem-solver by choice.


In [27]:
response = conv_chain.invoke(
    {"question": "What does he do?"},
    config={"configurable": {"session_id": "user-1"}}
)

print(response)


He designs impactful systems and leads innovative projects.


In [28]:
response = conv_chain.invoke(
    {"question": "What are his skills?"},
    config={"configurable": {"session_id": "user-1"}}
)

print(response)


His skills include technical expertise in full-stack development, with a focus on frontend and backend web application development using tools like React, Vite, Tailwind CSS, FastAPI, MySQL, and AI integration with Gemini API and LLM-based chat systems. Additionally, he has leadership abilities and business insight, as demonstrated by his role as the Public Relations Officer for FUNIEC, where he honed skills in messaging, stakeholder engagement, and project promotion.
